In [ ]:
# Video utilities

In [ ]:
!pip install open-cv

In [4]:
#export

from pathlib import Path
from fastai.vision.all import *

import cv2

def sample_frames(video_path, sample_interval, skip=0):
    cap = cv2.VideoCapture(str(video_path))
    i = -skip
    while cap.isOpened():
        ret, frame = cap.read()
        if ret == False:
            break
        if i % sample_interval == 0:
            yield frame
        i += 1

    cap.release()
    cv2.destroyAllWindows()

class Framer:
    def __init__(self, dst, sample_interval, target_size=None):
        self.dst = Path(dst)
        self.sample_interval = sample_interval
        self.target_size = target_size
    
    def __call__(self, video_path):
        out_dir = self.dst / Path(video_path).stem
        out_dir.mkdir(exist_ok=True, parents=True)
        for i, frame in enumerate(sample_frames(video_path, self.sample_interval)):
            filepath = out_dir / f"{i}.jpg"
            if self.target_size:
                frame = cv2.resize(frame, target_size, interpolation = cv2.INTER_CUBIC)
            cv2.imwrite(str(filepath), frame)
        
    def new(self, *, dst=None, sample_interval=None, target_size=None):
        dst = self.dst if dst is None else dst
        sample_interval = self.sample_interval if sample_interval is None else sample_interval
        target_size = self.target_size if target_size is None else target_size
        return self.__class__(dst=dst, sample_interval=sample_interval, target_size=target_size)


In [5]:
def extend_short_videos(directory, dry_run=False):
    frames = sorted(get_files(directory, '.jpg'), key=lambda p: int(p.stem))
    if not len(frames) < 50:
        return
    print(directory)
    for i in range(int(frames[-1].stem) + 1, 50):
        j = 49 - i
        src = frames[j]
        dst = src.parent / f"{i}.jpg"
        print(f"\tCopying {src} to {dst}")
        if not dry_run:
            shutil.copy(src, dst)

In [9]:
# samples frames from videos
# most of the videos are 10s longs but a few are 5s long, 
# so we extend them to 10s by rewinding it backward.
from tqdm import tqdm

k = 4
target_size = (1920 // k, 1080 // k)
target_fps = 5

dataset_dir = Path('path/to/video-dataset') # change here
frame_dir = dataset_dir.parent / 'frames' / f'{target_size[0]}x{target_size[1]}'
extensions = ['.mp4', '.avi']
for src_name, dst_name  in [('train-videos', 'train-frames'), ('test-videos', 'test-frames')]:
    dst = frame_dir / dst_name
    framify = Framer(dst, 30 // target_fps, target_size=target_size)
    video_paths = get_files(dataset_dir / src_name, extensions)
    for video_path in tqdm(video_paths):
        framify(video_path)
    # extend short videos
    for p in dst.glob('*'):
        if not p.is_dir():
            continue
        extend_short_videos(p, dry_run=False)

100%|██████████| 17/17 [00:27<00:00,  1.62s/it]


/Users/bdsaglam/knowledge/data-informatics/MMI-711/project/presentation/frames/480x270/train-frames/outside_bank_D3_D5_4
	Copying /Users/bdsaglam/knowledge/data-informatics/MMI-711/project/presentation/frames/480x270/train-frames/outside_bank_D3_D5_4/24.jpg to /Users/bdsaglam/knowledge/data-informatics/MMI-711/project/presentation/frames/480x270/train-frames/outside_bank_D3_D5_4/25.jpg
	Copying /Users/bdsaglam/knowledge/data-informatics/MMI-711/project/presentation/frames/480x270/train-frames/outside_bank_D3_D5_4/23.jpg to /Users/bdsaglam/knowledge/data-informatics/MMI-711/project/presentation/frames/480x270/train-frames/outside_bank_D3_D5_4/26.jpg
	Copying /Users/bdsaglam/knowledge/data-informatics/MMI-711/project/presentation/frames/480x270/train-frames/outside_bank_D3_D5_4/22.jpg to /Users/bdsaglam/knowledge/data-informatics/MMI-711/project/presentation/frames/480x270/train-frames/outside_bank_D3_D5_4/27.jpg
	Copying /Users/bdsaglam/knowledge/data-informatics/MMI-711/project/present

100%|██████████| 2/2 [00:02<00:00,  1.44s/it]
